In [5]:
import pandas as pd
import numpy as np
from statsmodels.tsa.api import VAR
from statsmodels.tsa.vector_ar.svar_model import SVAR
from statsmodels.tsa.filters.hp_filter import hpfilter
import matplotlib.pyplot as plt

# Step 1: Load the data
data = pd.read_excel('DATA.xlsx')
#data['Date'] = pd.PeriodIndex(data['Date'], freq='Q')
#data.set_index('Date', inplace=True)

# Select the sample period (1948Q1 to 1994Q4)
#data = data.loc['1948Q1':'1994Q4']

# Step 2: Define variables based on nchoice
nchoice = 2  # 1 for employment, 2 for hours
if nchoice == 1:
    nx = data['LHEM']
    labor = 'employment'
elif nchoice == 2:
    nx = data['LPMHU']
    labor = 'hours'

# Step 3: Compute variables
yx = data['GDPQ']
xx = yx / nx

# Create index numbers starting at 100
y = 100 + 100 * np.log(yx / yx.iloc[0])
n = 100 + 100 * np.log(nx / nx.iloc[0])
x = 100 + 100 * np.log(xx / xx.iloc[0])

# Compute first differences
dy = y.diff()
dn = n.diff()
dx = x.diff()

# Step 4: Handle the transformation of 'n' based on nint and difn
nint = 1  # 1 if 'n' is I(1), 0 if 'n' is I(0)
difn = 'yes'  # 'yes' to use first differences, 'no' to use detrended series

if nint == 0:
    # Detrend 'n' by regressing on a constant and trend
    trend = np.arange(len(n))
    n_trend = pd.DataFrame({'n': n, 'trend': trend})
    n_detrended = n_trend['n'] - np.polyval(np.polyfit(trend, n_trend['n'], 1), trend)
    dnz = n_detrended
else:
    # Use first differences
    dnz = dn

# If difn is 'no' and nint is 0, use the detrended 'n' for 'dn'
if nint == 0 and difn == 'no':
    dn = n_detrended

# Step 5: Prepare data for VAR
var_data = pd.DataFrame({'dx': dx, 'dnz': dnz}).dropna()

# Step 6: Estimate the reduced-form VAR
model = VAR(var_data)
results = model.fit(4)  # Using 4 lags as in the RATS code



c:\Users\joshu\miniconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


In [8]:

# Define the identification matrix for the SVAR model
A = np.array([[np.nan, 0], 
              [np.nan, np.nan]])

# Fit the SVAR model
svar_model = SVAR(var_data, svar_type='A', A=A)
svar_results = svar_model.fit()

# Proceed with structural shocks and impulse responses
epsilon = svar_results.resid
irf = svar_results.irf(10)

# Step 8: Obtain structural shocks and impulse responses
epsilon = svar_results.resid
nsteps = 13  # Number of periods for impulse responses
irf = svar_results.irf(nsteps)

# Step 9: Compute contributions of each shock (Historical Decomposition)
# Note: statsmodels does not have built-in historical decomposition, so we approximate it
# by filtering the shocks through the impulse responses

# Initialize arrays to store contributions
dx_tech = np.zeros(len(var_data))
dx_nontech = np.zeros(len(var_data))
dn_tech = np.zeros(len(var_data))
dn_nontech = np.zeros(len(var_data))

# Get structural impulse responses
irf_values = irf.irfs  # Shape: (nsteps+1, nvars, nvars)

# Loop over time to compute contributions
for t in range(4, len(var_data)):  # Start from index equal to the number of lags
    contrib_tech = np.zeros(2)
    contrib_nontech = np.zeros(2)
    for i in range(nsteps):
        if t - i >= 0:
            shock = epsilon.iloc[t - i]
            contrib_tech += irf_values[i, :, 0] * shock[0]
            contrib_nontech += irf_values[i, :, 1] * shock[1]
    dx_tech[t] = contrib_tech[0]
    dn_tech[t] = contrib_tech[1]
    dx_nontech[t] = contrib_nontech[0]
    dn_nontech[t] = contrib_nontech[1]

# Step 10: Compute conditional correlations
# Technology component
corr_tech = np.corrcoef(dn_tech[4:], dx_tech[4:])[0, 1]
# Non-technology component
corr_nontech = np.corrcoef(dn_nontech[4:], dx_nontech[4:])[0, 1]

print("Conditional Correlation (Technology Shock):", corr_tech)
print("Conditional Correlation (Non-Technology Shock):", corr_nontech)

# Step 11: Generate Figure 1 - Scatter plots
plt.figure(figsize=(12, 8))

# Original data
plt.subplot(3, 1, 1)
plt.scatter(dn[4:], dx[4:], alpha=0.5)
plt.title('Data')
plt.xlabel('dn')
plt.ylabel('dx')

# Technology component
plt.subplot(3, 1, 2)
plt.scatter(dn_tech[4:], dx_tech[4:], alpha=0.5)
plt.title('Technology Component')
plt.xlabel('dn (tech)')
plt.ylabel('dx (tech)')

# Non-Technology component
plt.subplot(3, 1, 3)
plt.scatter(dn_nontech[4:], dx_nontech[4:], alpha=0.5)
plt.title('Non-Technology Component')
plt.xlabel('dn (non-tech)')
plt.ylabel('dx (non-tech)')

plt.tight_layout()
plt.show()

# Step 12: Generate Figure 2 - Impulse Responses
irf.plot(orth=False, impulse='dx', response='dx', figsize=(12, 8))
plt.suptitle('Impulse Response of dx to Shocks', fontsize=16)
plt.show()

irf.plot(orth=False, impulse='dnz', response='dnz', figsize=(12, 8))
plt.suptitle('Impulse Response of dn to Shocks', fontsize=16)
plt.show()

# Additional plots for other variables can be generated similarly


c:\Users\joshu\miniconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


ValueError: zero-size array to reduction operation maximum which has no identity